<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Expert%20Mode/Mnist_Expert_Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

In [5]:
print(tf.__version__)

2.6.0


# Preparing Dataset
  

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0

11501568/11490434 [==============================] - 0s 0us/step


In [9]:
 x_train.shape

(60000, 28, 28)

In [10]:
x_train = x_train[...,tf.newaxis].astype('float32')
x_test = x_test[...,tf.newaxis].astype('float32')

In [11]:
x_train.shape

(60000, 28, 28, 1)

In [12]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Define Model

In [15]:
class MyModel(tf.keras.Model):

  def __init__(self,number_of_classes):
    super().__init__()
    self.conv1=Conv2D(32,(3,3),activation='relu')
    self.flatten=Flatten()
    self.fc1=Dense(128,activation='relu')
    self.fc2=Dense(number_of_classes, activation='softmax')

  def call(self,x):

    y = self.conv1(x)

    w = self.flatten(y)

    z = self.fc1(w)

    output = self.fc2(z)
     
    return output
  
model = MyModel(10)

In [16]:
 loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

 optimizer = tf.keras.optimizers.Adam()

In [18]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_loss')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_loss')

In [30]:
def train_step(images, y):
  with tf.GradientTape() as tape:
    y_pred = model(images, training=True)
    loss = loss_function(y, y_pred)

  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(y,y_pred)

In [31]:
def test_step(images, y):
  y_pred = model(images, training=False)
  loss = loss_function(y, y_pred)

  test_loss(loss)
  test_acc(y, y_pred)

In [32]:
from tqdm import tqdm

In [33]:
def train():
  epochs = 5

  for epoch in range(epochs):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()

    for images, labels in tqdm(train_data):
      train_step(images, labels)

    for images, labels in tqdm(test_data):
      test_step(images, labels)

    print('epoch:', epoch + 1)
    print('loss:', train_loss.result())
    print('accuracy:', train_acc.result())
    print('val loss:', test_loss.result())
    print('val accuracy:', test_acc.result())

# Fit

In [34]:
train()

  0%|          | 0/1875 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '
100%|██████████| 1875/1875 [00:41<00:00, 45.59it/s]


epoch: 1
loss: tf.Tensor(0.13839418, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9579833, shape=(), dtype=float32)
val loss: tf.Tensor(0.051397692, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.9848667, shape=(), dtype=float32)


100%|██████████| 1875/1875 [00:23<00:00, 80.32it/s]


epoch: 2
loss: tf.Tensor(0.044776816, shape=(), dtype=float32)
accuracy: tf.Tensor(0.98628336, shape=(), dtype=float32)
val loss: tf.Tensor(0.03450815, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.98915, shape=(), dtype=float32)


100%|██████████| 1875/1875 [00:41<00:00, 45.67it/s]


epoch: 3
loss: tf.Tensor(0.024143366, shape=(), dtype=float32)
accuracy: tf.Tensor(0.99226665, shape=(), dtype=float32)
val loss: tf.Tensor(0.013415097, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.99558336, shape=(), dtype=float32)


100%|██████████| 1875/1875 [00:23<00:00, 80.35it/s]


epoch: 4
loss: tf.Tensor(0.014977431, shape=(), dtype=float32)
accuracy: tf.Tensor(0.99506664, shape=(), dtype=float32)
val loss: tf.Tensor(0.00744355, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.9975833, shape=(), dtype=float32)


100%|██████████| 1875/1875 [00:23<00:00, 81.26it/s]

epoch: 5
loss: tf.Tensor(0.010906981, shape=(), dtype=float32)
accuracy: tf.Tensor(0.99635, shape=(), dtype=float32)
val loss: tf.Tensor(0.0036947704, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.99876666, shape=(), dtype=float32)
